In [50]:
u.user <- read.csv(file = 'u.user.csv', sep='|', header=T)
u.item <- read.csv(file = 'u.item.csv', sep='|', header=T)
u.data <- read.csv(file = 'u.data.csv', sep='|', header=T)
colMeans.sparse <- function(m) {colSums(m)/colSums(m>0)}
rowMeans.sparse <- function(m) {rowSums(m)/rowSums(m>0)}
means.sparse <- function(v) {sum(v)/sum(v>0)}

In [7]:
library(Matrix)
#library(dptyr)
#library(readr)

In [8]:
head(u.user)

id,age,gender,job,zip
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
6,42,M,executive,98101


In [9]:
head(u.data)

user.id,item.id,rating,timestamp
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596
298,474,4,884182806


In [10]:
head(u.item)

movie.id,movie.title,release.date,video.release.date,IMDb.URL,unknown,Action,Adventure,Animation,Children.s,...,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,NA,http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
v.u <- merge(u.data, u.user, by.x='user.id', by.y='id')

In [12]:
head(v.u)

user.id,item.id,rating,timestamp,age,gender,job,zip
1,1,5,874965758,24,M,technician,85711
1,23,4,875072895,24,M,technician,85711
1,223,5,876892918,24,M,technician,85711
1,171,5,889751711,24,M,technician,85711
1,16,5,878543541,24,M,technician,85711
1,73,3,876892774,24,M,technician,85711


In [13]:
q11 <- aggregate(v.u$rating, FUN=mean, by=list(v.u$job))


In [14]:
q11

Group.1,x
administrator,3.635646
artist,3.653380
doctor,3.688889
educator,3.670621
engineer,3.541407
entertainment,3.441050
executive,3.349104
healthcare,2.896220
homemaker,3.301003
lawyer,3.735316


In [15]:
# groub by groupe de 5

q11 <- aggregate(v.u$rating, FUN=mean, by=list(v.u$age))
q11

Group.1,x
7,3.767442
10,3.387097
11,2.925926
13,3.470825
14,3.375000
15,3.264484
16,3.331343
17,3.516165
18,3.696710
19,3.400968


In [16]:
## Cosinus entre un vecteur v et chaque colonne dela matrice m
cosinus.vm <- function(v,m) { n <- sqrt(colSums(m^2)); (v %*% m)/(n * sqrt(sum(v^2))) }

# Trouve les indexes des premiÃ¨res 'n' valeurs maximales d'une matrice
max.nindex <- function(m, n=5) {
    i <- order(m, decreasing=TRUE)
    return(i[1:n])
}
min.nindex <- function(m, n=5) {
    i <- order(m)
    return(i[1:n])
}

In [17]:
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
m <- as.matrix(m.sparse)
rownames(m) <- paste('u', 1:nrow(m), sep='')
colnames(m) <- paste('i', 1:ncol(m), sep='')

In [18]:
m[m==0] <- NA

In [19]:
ST_id <- u.item$movie.id[u.item$movie.title=="Star Trek V: The Final Frontier (1989)"]
ST_id

[1] 450

In [20]:
ST.cos <- cosinus.vm(m.sparse[,ST_id], m.sparse)
closestCos <- max.nindex(as.matrix(ST.cos), 11)[-1]
closestCos <- u.item$movie.title[closestCos]
closestCos

[1] Star Trek: The Motion Picture (1979)         
 [2] Star Trek VI: The Undiscovered Country (1991)
 [3] Star Trek III: The Search for Spock (1984)   
 [4] Star Trek IV: The Voyage Home (1986)         
 [5] Star Trek: The Wrath of Khan (1982)          
 [6] Stargate (1994)                              
 [7] Star Trek: Generations (1994)                
 [8] Die Hard 2 (1990)                            
 [9] Escape from New York (1981)                  
[10] Conan the Barbarian (1981)                   
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [21]:
colMeans(m, na.rm=TRUE)

i1       i2       i3       i4       i5       i6       i7       i8 
3.878319 3.206107 3.033333 3.550239 3.302326 3.576923 3.798469 3.995434 
      i9      i10      i11      i12      i13      i14      i15      i16 
3.896321 3.831461 3.847458 4.385768 3.418478 3.967213 3.778157 3.205128 
     i17      i18      i19      i20      i21      i22      i23      i24 
3.119565 2.800000 3.956522 3.416667 2.761905 4.151515 4.120879 3.448276 
     i25      i26      i27      i28      i29      i30      i31      i32 
3.443686 3.452055 3.105263 3.931159 2.666667 3.945946 3.629870 3.790123 
     i33      i34      i35      i36      i37      i38      i39      i40 
3.453608 2.714286 2.181818 2.153846 2.250000 3.008333 3.264368 2.894737 
     i41      i42      i43      i44      i45      i46      i47      i48 
3.081081 3.804054 3.000000 3.341772 4.050000 3.555556 3.601504 4.094017 
     i49      i50      i51      i52      i53      i54      i55      i56 
3.320988 4.358491 3.456790 3.769231 2.953125 3.240385 3.704698 4.060914 
     i57      i58      i59      i60      i61      i62      i63      i64 
4.000000 3.645714 4.060241 4.015625 3.864407 3.141732 3.097561 4.445230 
     i65      i66      i67      i68      i69      i70      i71      i72 
3.539130 3.549383 3.048544 3.417910 3.853583 3.661355 3.781818 3.193798 
     i73      i74      i75      i76      i77      i78      i79      i80 
3.468750 3.000000 3.000000 3.407407 3.278146 2.454545 4.044643 2.764706 
     i81      i82      i83      i84      i85      i86      i87      i88 
3.554545 3.720307 4.062500 3.000000 3.034483 3.940000 3.905797 3.539906 
     i89      i90      i91      i92      i93      i94      i95      i96 
4.138182 3.052632 3.587413 3.615385 3.723214 3.087591 3.812785 4.006780 
     i97      i98      i99     i100     i101     i102     i103     i104 
3.792969 4.289744 3.709302 4.155512 3.260274 3.129630 1.866667 1.400000 
    i105     i106     i107     i108     i109     i110     i111     i112 
2.567568 2.887324 3.500000 3.076923 3.430769 2.516129 3.485294 2.450000 
    i113     i114     i115     i116     i117     i118     i119     i120 
4.111111 4.447761 3.733333 3.824000 3.693122 3.215017 4.500000 2.238806 
    i121     i122     i123     i124     i125     i126     i127     i128 
3.438228 2.339623 3.234783 4.053476 3.557377 3.804124 4.283293 3.538462 
    i129     i130     i131     i132     i133     i134     i135     i136 
3.821705 2.260870 3.726316 4.077236 3.871345 4.292929 3.969112 4.123810 
    i137     i138     i139     i140     i141     i142     i143     i144 
3.900585 2.578947 2.780000 3.213115 3.500000 3.070175 3.765766 3.872428 
    i145     i146     i147     i148     i149     i150     i151     i152 
2.446154 2.500000 3.470270 3.203125 3.000000 3.828025 3.631902 3.634146 
    i153     i154     i155     i156     i157     i158     i159     i160 
3.785425 3.747126 3.102041 3.986486 3.692913 2.616667 3.227723 3.463768 
    i161     i162     i163     i164     i165     i166     i167     i168 
3.481818 3.556604 3.554348 3.589404 4.109375 4.120690 2.955224 4.066456 
    i169     i170     i171     i172     i173     i174     i175     i176 
4.466102 4.173554 3.876923 4.204360 4.172840 4.252381 3.817308 3.947183 
    i177     i178     i179     i180     i181     i182     i183     i184 
3.861314 4.344000 3.909502 4.045249 4.007890 3.951327 4.034364 3.431034 
    i185     i186     i187     i188     i189     i190     i191     i192 
4.100418 3.836653 4.186603 3.682353 4.106061 4.137097 4.163043 4.120690 
    i193     i194     i195     i196     i197     i198     i199     i200 
3.917197 4.058091 3.933555 3.916335 4.104603 4.007874 4.175758 3.825243 
    i201     i202     i203     i204     i205     i206     i207     i208 
3.516854 3.764286 3.868132 3.834286 3.992647 3.380000 3.818182 3.945000 
    i209     i210     i211     i212     i213     i214     i215     i216 
3.905759 3.930514 3.912621 3.456522 3.985075 3.350877 3.698113 3.910345 
    i217     i218     i219     i220     i221     i222     i223  

In [22]:
colSums(m.sparse)/colSums(m.sparse>0)

[1] 3.878319 3.206107 3.033333 3.550239 3.302326 3.576923 3.798469 3.995434
   [9] 3.896321 3.831461 3.847458 4.385768 3.418478 3.967213 3.778157 3.205128
  [17] 3.119565 2.800000 3.956522 3.416667 2.761905 4.151515 4.120879 3.448276
  [25] 3.443686 3.452055 3.105263 3.931159 2.666667 3.945946 3.629870 3.790123
  [33] 3.453608 2.714286 2.181818 2.153846 2.250000 3.008333 3.264368 2.894737
  [41] 3.081081 3.804054 3.000000 3.341772 4.050000 3.555556 3.601504 4.094017
  [49] 3.320988 4.358491 3.456790 3.769231 2.953125 3.240385 3.704698 4.060914
  [57] 4.000000 3.645714 4.060241 4.015625 3.864407 3.141732 3.097561 4.445230
  [65] 3.539130 3.549383 3.048544 3.417910 3.853583 3.661355 3.781818 3.193798
  [73] 3.468750 3.000000 3.000000 3.407407 3.278146 2.454545 4.044643 2.764706
  [81] 3.554545 3.720307 4.062500 3.000000 3.034483 3.940000 3.905797 3.539906
  [89] 4.138182 3.052632 3.587413 3.615385 3.723214 3.087591 3.812785 4.006780
  [97] 3.792969 4.289744 3.709302 4.155512 3.260274 3.129630 1.866667 1.400000
 [105] 2.567568 2.887324 3.500000 3.076923 3.430769 2.516129 3.485294 2.450000
 [113] 4.111111 4.447761 3.733333 3.824000 3.693122 3.215017 4.500000 2.238806
 [121] 3.438228 2.339623 3.234783 4.053476 3.557377 3.804124 4.283293 3.538462
 [129] 3.821705 2.260870 3.726316 4.077236 3.871345 4.292929 3.969112 4.123810
 [137] 3.900585 2.578947 2.780000 3.213115 3.500000 3.070175 3.765766 3.872428
 [145] 2.446154 2.500000 3.470270 3.203125 3.000000 3.828025 3.631902 3.634146
 [153] 3.785425 3.747126 3.102041 3.986486 3.692913 2.616667 3.227723 3.463768
 [161] 3.481818 3.556604 3.554348 3.589404 4.109375 4.120690 2.955224 4.066456
 [169] 4.466102 4.173554 3.876923 4.204360 4.172840 4.252381 3.817308 3.947183
 [177] 3.861314 4.344000 3.909502 4.045249 4.007890 3.951327 4.034364 3.431034
 [185] 4.100418 3.836653 4.186603 3.682353 4.106061 4.137097 4.163043 4.120690
 [193] 3.917197 4.058091 3.933555 3.916335 4.104603 4.007874 4.175758 3.825243
 [201] 3.516854 3.764286 3.868132 3.834286 3.992647 3.380000 3.818182 3.945000
 [209] 3.905759 3.930514 3.912621 3.456522 3.985075 3.350877 3.698113 3.910345
 [217] 3.158333 3.456140 3.171171 3.363636 3.729730 3.660274 4.198529 3.704545
 [225] 2.908257 3.355422 3.279503 3.815574 3.111111 3.447236 2.683099 3.207921
 [233] 3.290323 3.775000 2.847926 3.622222 3.710938 3.875000 3.513333 2.788462
 [241] 3.546875 3.991453 2.439394 3.166667 3.108333 3.935484 1.800000 3.675000
 [249] 3.246154 3.477157 4.260870 2.943038 3.692308 2.451613 3.360465 3.562500
 [257] 3.745875 3.803536 2.685185 2.574803 2.558140 3.712121 2.157895 2.742574
 [265] 3.863436 2.714286 3.444444 3.792157 3.926984 3.595588 3.232227 4.262626
 [273] 3.569507 3.500000 4.011194 3.697987 3.464789 3.233333 3.285714 3.188235
 [281] 3.143836 3.685345 3.717514 3.160622 4.265432 3.656965 3.589744 3.441423
 [289] 2.980695 3.114583 3.370079 3.666667 3.802721 3.156701 3.402597 3.333333
 [297] 3.960000 3.768041 2.931507 3.631090 3.304348 4.161616 3.686567 3.536913
 [305] 3.643678 3.947917 3.515957 2.966667 2.857143 3.600000 3.680000 3.162500
 [313] 4.245714 1.000000 4.100000 4.196429 3.911765 4.466443 3.273810 4.050000
 [321] 3.130178 3.087156 2.933333 3.136000 2.546875 3.360000 3.377143 3.423729
 [329] 3.333333 3.024390 3.539823 3.461538 3.593625 3.359375 2.761905 2.627907
 [337] 3.000000 2.659341 3.255319 3.587302 3.090909 2.923077 3.096774 3.654545
 [345] 3.400000 3.642857 3.510949 3.296296 2.903226 3.146341 2.700000 2.115385
 [353] 2.428571 3.486111 2.926829 3.381443 4.291667 2.615385 3.500000 3.200000
 [361] 3.500000 2.821429 2.723404 2.675676 3.208333 3.340426 3.458824 1.903226
 [369] 2.636364 2.820513 3.164179 3.529412 2.897436 2.272727 1.956522 2.833333
 [377] 2.153846 3.722772 2.976744 3.336207 3.550000 3.594595 2.064516 2.782609
 [385] 3.562500 2.816092 3.384615 2.392857 3.444444 3.200000 2.745763 3.544118
 [393] 3.411458 3.333333 2.589286 2.814815 2.583333 2.076923 3.303371 2.722222
 [401] 2.802632 3.464706 3.427861 3.673267 3.313953 2

In [23]:
cor.vm <- function(v,m) {v_a <-sum(v)/sum(v>0); v_i <- colMeans.sparse(m); n <- sqrt(colSums((t(t(m)-v_i))^2)); (t(v-v_a) %*% (t(t(m)-v_i)))/(n * sqrt(sum((v-v_a)^2))) }
ST.cor <- cor.vm(m.sparse[,ST_id], m.sparse)
closestCor <- max.nindex(as.matrix(ST.cor), 11)[-1]
closestCor <- u.item$movie.title[closestCor]
closestCor

[1] Robocop 3 (1993)                                 
 [2] Jerky Boys, The (1994)                           
 [3] Across the Sea of Time (1995)                    
 [4] S.F.W. (1994)                                    
 [5] Tough and Deadly (1995)                          
 [6] Open Season (1996)                               
 [7] Blood For Dracula (Andy Warhol's Dracula) (1974) 
 [8] Vampire in Brooklyn (1995)                       
 [9] Fire on the Mountain (1996)                      
[10] Police Story 4: Project S (Chao ji ji hua) (1993)
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [24]:
v<- m.sparse[,450]
v_a <-sum(v)/sum(v>0)
v_i <- colMeans.sparse(m.sparse) # why columns and not row
m.center <- t(t(m.sparse) - v_i)*(m.sparse>0)
v.center <- (v-v_a)*(v>0)
n <- colSums(m.center^2)
ST.cor <-(v.center %*% m.center)/sqrt(n * sum(v.center^2))
closestCor <- max.nindex(as.matrix(ST.cor), 11)[-1]
closestCor <- u.item$movie.title[closestCor]
closestCor

[1] Star Trek: The Motion Picture (1979)      
 [2] Star Trek III: The Search for Spock (1984)
 [3] Robocop 3 (1993)                          
 [4] Just Cause (1995)                         
 [5] Three Musketeers, The (1993)              
 [6] Star Trek: Generations (1994)             
 [7] Money Train (1995)                        
 [8] House Party 3 (1994)                      
 [9] Mr. Magoo (1997)                          
[10] S.F.W. (1994)                             
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [25]:
ST.cor[450]

[1] 1

In [26]:
ST.cor <- cor.vm(m[,ST_id], m)
closestCor <- max.nindex(as.matrix(ST.cor), 11)

In [27]:
ST.cos[450]

[1] 1

In [28]:
#essaie
temp <-(((m.sparse[,450] - m.sparse))^2)*((m.sparse[,450]>0- (m.sparse>0)))^2
new.dist <- sqrt(colSums((temp>0)))

In [36]:
voisins <- 21
# centrer selon lutilisateur les notes des films quil a donne

u.mean <-rowMeans.sparse(m.sparse)
m.center <- as.matrix((m.sparse-u.mean)*(m.sparse>0))
m.center[m.center ==0] <- NA
distance.na.450.center <- sqrt(colSums((m.center[,450] - m.center)^2, na.rm=T)) # ignore les valeurs manquantes


votes.communs <- (colSums((m.sparse[,450] * m.sparse) > 0))
distance.na.450.center[votes.communs<40] <- Inf


# prendre films si plus ou = 5 votes en commun avec 450


In [37]:
closest.euc <- min.nindex(distance.na.450.center,voisins)[-1]
closestEuc <- u.item$movie.title[closest.euc]
closestEuc

[1] Star Trek: The Motion Picture (1979)         
 [2] Star Trek III: The Search for Spock (1984)   
 [3] Mission: Impossible (1996)                   
 [4] Stargate (1994)                              
 [5] Speed (1994)                                 
 [6] Die Hard 2 (1990)                            
 [7] Hunt for Red October, The (1990)             
 [8] Independence Day (ID4) (1996)                
 [9] Highlander (1986)                            
[10] Jurassic Park (1993)                         
[11] Top Gun (1986)                               
[12] True Lies (1994)                             
[13] Rock, The (1996)                             
[14] Apollo 13 (1995)                             
[15] Star Trek VI: The Undiscovered Country (1991)
[16] Groundhog Day (1993)                         
[17] Batman (1989)                                
[18] Star Trek IV: The Voyage Home (1986)         
[19] Aladdin (1992)                               
[20] Indiana Jones and the Last Crusade (1989)    
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [31]:
wcos.st <- cosinus.vm(m.sparse[,ST_id], m.sparse[,closest.euc])
wcos.st

1 x 20 Matrix of class "dgeMatrix"
     [,1]      [,2]      [,3]      [,4]      [,5]      [,6]     [,7]      [,8]
[1,]    1 0.6080518 0.5455677 0.2665322 0.4540497 0.2916639 0.398426 0.3381603
         [,9]   [,10]     [,11]     [,12]     [,13]     [,14]     [,15]
[1,] 0.287455 0.36823 0.3295916 0.3476904 0.3711141 0.2355901 0.2497198
        [,16]     [,17]     [,18]     [,19]     [,20]
[1,] 0.553322 0.3060916 0.3377897 0.5144135 0.2959786

In [32]:
esperence <- function(v.item, w, idx, m) {
    v_1 <-sum(v.item)/sum(v.item>0);
    v_i <- apply(m[, 1, function(x) sum(x)/sum(x>0))
    v_i <- colMeans.sparse(m);
    n <- sqrt(colSums((t(t(m)-v_i))^2));
    (t(v-v_a) %*% (t(t(m)-v_i)))/(n * sqrt(sum((v-v_a)^2))) }


ERROR: Error in parse(text = x, srcfile = src): <text>:3:52: unexpected ')'
2:     v_1 <-sum(v.item)/sum(v.item>0);
3:     v_i <- apply(m[, 1, function(x) sum(x)/sum(x>0))
                                                      ^


In [79]:
idx<- closest.euc
m.sparse.t <- t(m.sparse)
v.item <- m.sparse.t[1,]
# mean of item i
v_1 <-sum(v.item)/sum(v.item>0);
# return means for each item
v_i <- apply(m.sparse.t[idx,], 1, FUN=means.sparse)
temp = m.sparse.t[idx,8]
temp.0 = temp>0
#diff between v_i et b_ij
temp1 <- (temp-v_i[1])*temp.0
# numerator
num <- temp1 %*% t(wcos.st)
# denumerator
denum <- sum(abs(wcos.st)*temp.0)
# predicted value
I_ij <- v_1 + num/denum


1 x 1 Matrix of class "dgeMatrix"
         [,1]
[1,] 2.175832

[1] 2.917267

1 x 1 Matrix of class "dgeMatrix"
         [,1]
[1,] 4.624165

[1] 3.878319

In [57]:
idx[1]

[1] 449